# Preparation

## Authentication

Before requests to orchestration can be issued, we need to provide authentication details to the SDK. This can be done either via a configuration file or via the environment. Make sure to check out the [SAP generative AI hub SDK project description](https://pypi.org/project/generative-ai-hub-sdk/) for more details. Below you will find an example for authenticating via environment variables using this very notebook.

> **WARNING:**
> Below code should never be used in production scenarios and is only for the purpose of illustrating which environment variables to use!
> Credentials should never be defined in code!

In [1]:
import json

with open('../aicore_proxy_participant_credentials_ai180p104road.json', 'r') as file:
    conn_data = json.load(file)

In [ ]:
conn_data

In [3]:
import os

os.environ["AICORE_AUTH_URL"] = conn_data["python"]["AICORE_AUTH_URL"]

In [ ]:
os.environ["AICORE_AUTH_URL"]

In [5]:
import os

os.environ["AICORE_AUTH_URL"] = conn_data["python"]["AICORE_AUTH_URL"]
os.environ["AICORE_BASE_URL"] = conn_data["python"]["AICORE_BASE_URL"]
os.environ["AICORE_CLIENT_ID"] = conn_data["python"]["AICORE_CLIENT_ID"]
os.environ["AICORE_CLIENT_SECRET"] = conn_data["python"]["AICORE_CLIENT_SECRET"]
os.environ["AICORE_ORCHESTRATION_DEPLOYMENT_URL"] = conn_data["python"]["AICORE_ORCHESTRATION_DEPLOYMENT_URL"]
os.environ["AICORE_RESOURCE_GROUP"] = conn_data["python"]["AICORE_RESOURCE_GROUP"]

You will need to authenticate in every exercise.

# Content Filtering

Orchestration also supports content filtering to moderate input and output. For each, multiple filters can be applied that remove harmful content from texts. Depending on the filter, sensitivity of various types can be configured.

## Input Filter

Input filtering is handy for blocking out harmful content form e.g. user input. To make use of input filtering we first up need to define a basic pipeline again. We use a simple prompt template, that just passes text provided as a parameter to an LLM.

In [6]:
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue

llm = LLM(
    name="gemini-1.5-flash",
    version="latest",
    parameters={"max_tokens": 256, "temperature": 0.2},
)

template = Template(messages=[UserMessage("{{?text}}")])

Next up we will configure an AzureContentFilter using the corresponding SDK primitives.

In [ ]:
from gen_ai_hub.orchestration.models.content_filter import AzureContentFilter

content_filter = AzureContentFilter(
    hate=0,
    sexual=0,
    self_harm=0,
    violence=0,
)

**Note:** The lower the sensitivity value the higher the sensitivity. Zero (0) corresponds to the highest degree of content moderation. For further information please check out: https://learn.microsoft.com/en-us/azure/ai-services/content-safety/concepts/harm-categories?tabs=warning

Now that we have all modules defined, the only thing left to do is plugging everything together.

In [8]:
from gen_ai_hub.orchestration.service import OrchestrationService
from gen_ai_hub.orchestration.models.config import OrchestrationConfig

config = OrchestrationConfig(
    template=template,
    llm=llm,
    input_filters=[
        content_filter,
    ],
)

orchestration_service = OrchestrationService(
    api_url=os.environ["AICORE_ORCHESTRATION_DEPLOYMENT_URL"],
    config=config,
)

If the content filter detects a violation when performing an inference an error of type `OrchestrationError` will be raised.

In [ ]:
from gen_ai_hub.orchestration.exceptions import OrchestrationError

try:
    result = orchestration_service.run(
        template_values=[
            TemplateValue(
                name="text",
                value="This is a hateful text!",
            ),
        ]
    )
except OrchestrationError as error:
    print(error.message)

The filter can be cleared by adjusting the prompt.

In [ ]:
result = orchestration_service.run(
    template_values=[
        TemplateValue(
            name="text",
            value="This is a peaceful text!",
        )
    ]
)
print(result.orchestration_result.choices[0].message.content)

## Output Filter

Similarly, also LLM output can be filtered. We will just use our already created filter and apply it to the LLM output within the configuration.

In [11]:
orchestration_service.config.output_filters = [content_filter]

Now let's try out if the filter works by prompting for an inherently violent lyrics from Johann Wolfgang von Goethe. Even though the version recited by the LLM might differ from the most common version, it should contain some form of suggested violence.

In [ ]:
result = orchestration_service.run(
    template_values=[
        TemplateValue(
            name="text",
            value='Provide the lyrics of the ballad "Erlkönig" from Johann Wolfgang von Goethe in German and English.',
        )
    ]
)
print(result.orchestration_result.choices[0].message.content)

**Note:** The behavior will differ from the input filter. Instead of raising an error, the returned content will be empty.

Feel free to try around with AzureContentFilter settings. If you set sensitivity for violence to 4 you will be able to read the lyrics of Erlkönig.

# Summary

In this exercise you learned how content filtering can be applied to input and output using orchestration. Now let's combine capabilities into a more complex scenario. Continue to [Exercise 3 - Orchestration Chatbot](./ex3.ipynb).